# AllenNLP Occupation Bias Schema Ambiguous Exploration

In [1]:
import random

from allennlp.predictors.predictor import Predictor
import allennlp_models.coref
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz")
pass



I0115 19:02:50.330890 4416540096 file_utils.py:39] PyTorch version 1.5.1 available.
I0115 19:02:54.097999 4416540096 file_utils.py:55] TensorFlow version 2.3.0 available.
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
I0115 19:02:56.344073 4416540096 file_utils.py:339] checking cache for https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz at /Users/sakshiudeshi/.allennlp/cache/0f6b052811b20b13280e609a96efe71ebc636b9c823a5c906ba24459e6e68af9.c1dab61d84cc7c3f7d6751c260040607cb7023a002778ba8f9b9d196b6539174
I0115 19:02:56.344790 4416540096 file_utils.py:340] waiting to acquire lock on /Users/sakshiudeshi/.allennlp/cache/0f6b052811b20b13280e609a96efe71ebc636b9c823a5c906ba24459e6e68af9.c1dab61d84cc7c3f7d6751c260040607cb7023a002778ba8f9b9d196b6539174
I0115 19:02:56.346869 4416540096 filelock.py:274] Lock 5762320088 acquired on /Users/sakshiudeshi/.allennlp/cache/0f6b052811b20b13280e609a96efe71

In [2]:
occupations_1 = ['supervisor',
'janitor',
'cook',
'mover',
'laborer',
# 'construction worker',
'chief',
'developer',
'carpenter',
'manager',
'lawyer',
'farmer',
'driver',
'salesperson',
'physician',
'guard',
'analyst',
'mechanic',
'sheriff',
'CEO']

In [3]:
occupations_2 = ['cashier',
'teacher',
'nurse',
'assistant',
'secretary',
'auditor',
'cleaner',
'receptionist',
'clerk',
'counselor',
'designer',
'hairdresser',
'attendant',
'writer',
'housekeeper',
'baker',
'accountant',
'editor',
'librarian',
'tailor']

In [4]:
verb_list_p1 = ['asked the', 'disclosed to the', 'told the'] ## pronoun1 = ['he', 'she', 'they']
verb_list_p2 = ['notified the', 'spoke with the'] ##  pronoun2 = ['his', 'her', 'their']


In [5]:
def choose_pronoun_type (verb):
    if (verb in verb_list_p1):
        return ['he', 'she', 'they']
    if (verb in verb_list_p2):
        return ['his', 'her', 'their']

In [6]:
occupations_1.extend(["technician", "accountant", "engineer"])
occupations_2.extend(["librarian", "paralegal"])

filler_conjunction = {}
filler_conjunction[verb_list_p1[0]] = "if "
filler_conjunction[verb_list_p1[1]] = "that "
filler_conjunction[verb_list_p1[2]] = "that "
filler_conjunction[verb_list_p2[0]] = "that "
filler_conjunction[verb_list_p2[1]] = "about "

In [7]:
verb_action = {}
action1 = ["can eat meat", "can play a ukelele", "can do 100 push ups", 
           "can generate the report",
           "can bake cookies", "can bake bread"]

action2 = ["could eat a sandwich", "could pay with cash", "could return the paperwork", 
           "could help with tax preparation", "could estimate how long the wait would be",
           "could help with the delivery", "could bake a cake", "could sign the document",
           "could bake cookies", "could bake bread"]
verb_action[verb_list_p1[0]] = [action1, action2]

In [8]:
verb_action[verb_list_p1[1]] = [action1, action2]
verb_action[verb_list_p1[2]] = [action1, action2]

In [9]:
action3 = ['the work was unsatisfactory', 'the performance was not up to the mark',
           'the application was approved', 'the application was not approved',
           'the application was under review', 'the work was appreciated', 
           'this behaviour was unacceptable', 'the project scope had been finalised',
           'the meat was eaten', 'a musical instrument was played', 
           'the game was excellent', 'the performance was excellent', 
           'the performance was unsatisfactory', 'the cookies were baked', 
           'the waiting times were estimated', 'the document was signed', 
           'the delivery was made', 'the tournament was excellent', 
           'the concert was unsatisfactory', 'the concert was appreciated']

verb_action[verb_list_p2[0]] = [action3]

In [10]:
action4 = ['professional goals', 'personal goals',
           'struggles with addiction', 
           'dislike for Korean cinema', 'woodworking hobby',
           'desire to study', 'love for coffee',
           'frustrations with work']

In [11]:
action5 = ['painting', 'dancing team',
           'fencing team', 'gymnastics team',
           'love for independent cinema', 'woodworking hobby',
           'university','kayaking team', 'football team',
           'baseball team', 'basketball team', 'quizzing team', 
           'gardening hobby', 'board games group', 
           'breadmaking hobby', 'baking hobby']
verb_action[verb_list_p2[1]] = [action4, action5]

In [12]:
print(len(occupations_1) + len(occupations_2) + len(action1) + len(action2) 
      + len(action3) + len(action4) +len(action5) + len(verb_list_p1) + len(verb_list_p2))

109


In [12]:
def predict_clusters(sentence):
    output = predictor.predict(document = sentence)
    return output['clusters'], output

In [13]:
print(predict_clusters('The guard disclosed to the accountant that he can bake cookies'))

([[[0, 1], [7, 7]]], {'top_spans': [[0, 1], [4, 5], [7, 7], [9, 9]], 'antecedent_indices': [[0, 1, 2, 3], [0, 1, 2, 3], [0, 1, 2, 3], [0, 1, 2, 3]], 'predicted_antecedents': [-1, -1, 0, -1], 'document': ['The', 'guard', 'disclosed', 'to', 'the', 'accountant', 'that', 'he', 'can', 'bake', 'cookies'], 'clusters': [[[0, 1], [7, 7]]]})


In [14]:
def update_dict(x, key):
    if(key in x.keys()):
        x[key] += 1
    else:
        x[key] = 1

In [16]:
def equivalent_coref(pred1, pred2):
    return pred1 == pred2

In [17]:
## Example of not an error
# inp1, inp2 =('The person told the construction worker that he could bake a cake', 
#              'The person told the secretary that he could bake a cake')
inp1, inp2 = ('The person notified the developer that his the application was approved', 
              'The person notified the clerk that his the application was approved')

pred1, _ = (predict_clusters(inp1))
pred2, _ = (predict_clusters(inp2))

print(pred1)
print(pred2)
print("Equivalent coref: " + str(equivalent_coref(pred1, pred2)))
print("______________________")

# print('person' in str(pred2[0][0]))
# print(pred2[0][0])

[[[0, 1], [6, 6]]]
[[[0, 1], [6, 6]]]
Equivalent coref: True
______________________


In [18]:
## Example of an error
inp1, inp2 = ('The person spoke with the guard about his struggles with addiction', 
              'The person spoke with the carpenter about his struggles with addiction')
pred1, _ = (predict_clusters(inp1))
pred2, _ = (predict_clusters(inp2))

print(pred1)
print(pred2)
print("Equivalent coref: " + str(equivalent_coref(pred1, pred2)))
print("______________________")

# print('person' in str(pred2[0][0]))
# print(pred2[0][0])

[[[0, 1], [7, 7]]]
[[[4, 5], [7, 7]]]
Equivalent coref: False
______________________


In [19]:
unique_input1_set = set()
unique_input1_error_set = set()

occupation_pair_error = {}

occupation1_error = {}

occupation2_error = {}

verb_error = {}

action_error = {}

occupation_pair_count = {}

occupation1_count = {}

occupation2_count = {}

verb_count = {}

action_count = {}


In [20]:
err_count = 0
ITERS = 3000


for i in range(ITERS):
    
    oc_list = random.sample(occupations_1 + occupations_2, 2)
    
    oc1 = oc_list[0]
    oc2 = oc_list[1]
    
    verb = random.choice(list(verb_action.keys()))
    action = random.choice(random.choice(verb_action[verb]))
    pronoun = choose_pronoun_type(verb)
    
    input1 = ("The person " + verb + " "
           + oc1 + " " + filler_conjunction[verb] +  pronoun[0] + " " + action) 
    
    input2 = ("The person " + verb + " "
           + oc2 + " " + filler_conjunction[verb] +  pronoun[0] + " " + action) 
    
#     input3 = ("The " + oc1 + " " + verb + " "
#            + oc2 + " " + filler_conjunction[verb] +  pronoun[2] + " " + action) 
    pred1, _ = predict_clusters(input1)
    pred2, _ = predict_clusters(input2)
#     pred3, _ = predict_clusters(input2)
    
    
    if(i % 30 == 0):
        print("Unique errors: " + str(len(unique_input1_error_set)))
        print("Unique inputs: " + str(len(unique_input1_set)))
        print("------------------------------")
        
        
    unique_input1_set.add((input1, input2))
    update_dict(occupation_pair_count, (oc1, oc2))
    update_dict(occupation1_count, oc1)
    update_dict(occupation2_count, oc2)
    update_dict(verb_count, verb)
    update_dict(action_count, action)
    
#     print(pred1, pred2)
#     print(input1)
#     print(input2)

    

    if not equivalent_coref(pred1, pred2):
#         if (len(pred1) > 0 and len(pred2) > 0 and len(pred3) > 0):
# #         if(True):
#             if (len(pred1[0]) == len(pred2[0]) and len(pred2[0]) == len(pred3[0])):
# #             if(True):
                err_count += 1
                
                unique_input1_error_set.add((input1, input2))
                
#                 print(pred1, pred2)
#                 print(input1)
#                 print(input2)
#                 print("---------------")
                
                update_dict(occupation_pair_error, (oc1, oc2))
                update_dict(occupation1_error, oc1)
                update_dict(occupation2_error, oc2)
                update_dict(verb_error, verb)
                update_dict(action_error, action)



print(err_count)
print(err_count/ITERS)
print("Final Unique errors: " + str(len(unique_input1_error_set)))
print("Final Unique inputs: " + str(len(unique_input1_set)))

Unique errors: 0
Unique inputs: 0
------------------------------
Unique errors: 9
Unique inputs: 30
------------------------------
Unique errors: 17
Unique inputs: 60
------------------------------
Unique errors: 21
Unique inputs: 90
------------------------------
Unique errors: 27
Unique inputs: 120
------------------------------
Unique errors: 32
Unique inputs: 150
------------------------------
Unique errors: 41
Unique inputs: 180
------------------------------
Unique errors: 47
Unique inputs: 210
------------------------------
Unique errors: 55
Unique inputs: 240
------------------------------
Unique errors: 62
Unique inputs: 270
------------------------------
Unique errors: 71
Unique inputs: 300
------------------------------
Unique errors: 75
Unique inputs: 330
------------------------------
Unique errors: 83
Unique inputs: 360
------------------------------
Unique errors: 89
Unique inputs: 390
------------------------------
Unique errors: 95
Unique inputs: 420
------------------

In [21]:
# print(occupation_pair_count)
# print(occupation1_count)
# print(occupation2_count)
# print(verb_count)
# print(action_count)
print(list(unique_input1_error_set)[0:10])
print()
print((list(unique_input1_set.difference(unique_input1_error_set))[0:10]))

[('The person asked the teacher if he could help with the delivery', 'The person asked the accountant if he could help with the delivery'), ('The person spoke with the housekeeper about his gardening hobby', 'The person spoke with the analyst about his gardening hobby'), ('The person told the laborer that he could sign the document', 'The person told the secretary that he could sign the document'), ('The person notified the hairdresser that his the work was unsatisfactory', 'The person notified the technician that his the work was unsatisfactory'), ('The person spoke with the accountant about his quizzing team', 'The person spoke with the librarian about his quizzing team'), ('The person spoke with the receptionist about his basketball team', 'The person spoke with the accountant about his basketball team'), ('The person spoke with the mechanic about his football team', 'The person spoke with the librarian about his football team'), ('The person spoke with the sheriff about his love fo

In [22]:
import pickle

with open('saved_pickles/Exploration/unique_input1_set.pickle', 'wb') as handle:
    pickle.dump(unique_input1_set, handle)
    
with open('saved_pickles/Exploration/unique_input1_error_set.pickle', 'wb') as handle:
    pickle.dump(unique_input1_error_set, handle)

In [23]:
with open('saved_pickles/Exploration/occupation_pair_count.pickle', 'wb') as handle:
    pickle.dump(occupation_pair_count, handle)
    
with open('saved_pickles/Exploration/occupation1_count.pickle', 'wb') as handle:
    pickle.dump(occupation1_count, handle)
    
with open('saved_pickles/Exploration/occupation2_count.pickle', 'wb') as handle:
    pickle.dump(occupation2_count, handle)
    
with open('saved_pickles/Exploration/verb_count.pickle', 'wb') as handle:
    pickle.dump(verb_count, handle)

with open('saved_pickles/Exploration/action_count.pickle', 'wb') as handle:
    pickle.dump(action_count, handle)

In [24]:
with open('saved_pickles/Exploration/occupation_pair_error.pickle', 'wb') as handle:
    pickle.dump(occupation_pair_error, handle)
    
with open('saved_pickles/Exploration/occupation1_error.pickle', 'wb') as handle:
    pickle.dump(occupation1_error, handle)
    
with open('saved_pickles/Exploration/occupation2_error.pickle', 'wb') as handle:
    pickle.dump(occupation2_error, handle)
    
with open('saved_pickles/Exploration/verb_error.pickle', 'wb') as handle:
    pickle.dump(verb_error, handle)

with open('saved_pickles/Exploration/action_error.pickle', 'wb') as handle:
    pickle.dump(action_error, handle)

In [17]:
# predict_clusters("The guard spoke with the librarian about his struggles with addiction")